# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import make_column_selector as selector
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD
from IPython.display import display, Markdown

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Configuration
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "logon_organization"
FEATURE_SETS  = [
    ["activity", "hour"],
    ["activity", "weekday"],
    ["activity", "hour", "weekday"],
    ["activity", "hour_sin", "hour_cos"],
    ["activity", "weekday_sin", "weekday_cos"],
    ["activity", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def train_classifier(df, classifier, features, params, n_iter=10):
    # Create a dataframe where the results of the iterations will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a clone of the classifier
        try:
            classifier = clone(classifier)
        except:
            pass

        # Update the parameters of the classifier according to the grid search
        classifier.set_params(**params)
        
        # Set the random state attribute of the classifier (if it has one)
        try:
            classifier.set_params(**{"random_state": randint(0, 2**32)})
        except Exception:
            pass

        # Define the transformers that do the rest of the preprocessing (scaling, encoding)
        numeric_transformer = Pipeline(steps=[
            ("scaler", RobustScaler())
        ])
        categorical_transformer = Pipeline(steps=[
            ("ohe", OneHotEncoder())
        ])

        # Filter the dataframe according to the features 
        df_filtered = df[features].copy()

        # Create a pipeline that performs the feature selection and scaling
        pipe = Pipeline([
            ("column_transformer", ColumnTransformer(
                transformers=[
                    ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                    ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                ]
            )),
            ("classifier", classifier)
        ])
       
        # Benchmark the training
        start_training = timer()
        pipe.fit(df_filtered)
        end_training = timer()

        # Benchmark the inference
        start_inference = timer()
        pipe.predict(df_filtered)
        end_inference = timer()

        # Calculate different evaluation metrics
        y_true = df["threat"]
        y_pred = pipe.named_steps["classifier"].labels_
        scores = pipe.named_steps["classifier"].decision_scores_
        recall = recall_score(y_true, y_pred)
        precision = precision_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred)
        cm = confusion_matrix(y_true, y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(y_true, scores)
            p_auc_10 = roc_auc_score(y_true, scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(y_true, scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(y_true, scores, max_fpr=0.3)
        except ValueError:
            auc = -1

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": end_training - start_training,
                "inference_time": end_inference - start_inference,
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": y_true,
                "y_pred": y_pred,
                "scores": scores
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, features, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.features = features
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None
        

    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, features in enumerate(FEATURE_SETS, start=1):

        display(Markdown(f"# {i}/{len(FEATURE_SETS)} - Features: {', '.join(features)}"))

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=df,
            classifier=classifier,
            features=features,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "logon"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 24 different hyperparameter combinations
 Training Time: 0.8650    Inference Time: 0.4884    pAUC: 0.6813 ± 0.0495   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.3918    Inference Time: 2.1843    pAUC: 0.7338 ± 0.0043   Recall: 0.0596 ± 0.0254   TN: 846287   FP: 8374    FN: 183   TP: 15    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 19.5097   Inference Time: 9.2635    pAUC: 0.7391 ± 0.0039   Recall: 0.0717 ± 0.0074   TN: 848129   FP: 6532    FN: 182   TP: 16    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 38.1729   Inference Time: 18.2208   pAUC: 0.7378 ± 0.0028   Recall: 0.0717 ± 0.0074   TN: 848129   FP: 6532    FN: 182   TP: 16    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.8393    Inference Time: 0.4823    pAUC: 0.6803 ± 0.0494   Recall: 0.0576 ± 0.0348   TN: 847809   FP: 6852    FN: 180   TP: 18    Params: {'max_sample

# 2/6 - Features: activity, weekday

Testing 24 different hyperparameter combinations
 Training Time: 0.7678    Inference Time: 0.4471    pAUC: 0.5207 ± 0.0126   Recall: 0.0162 ± 0.0198   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 3.6401    Inference Time: 1.8048    pAUC: 0.5074 ± 0.0008   Recall: 0.0162 ± 0.0198   TN: 846374   FP: 8287    FN: 190   TP: 8     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 16.5281   Inference Time: 7.7968    pAUC: 0.5065 ± 0.0003   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 32.5103   Inference Time: 15.3113   pAUC: 0.5065 ± 0.0003   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.7782    Inference Time: 0.4564    pAUC: 0.5138 ± 0.0082   Recall: 0.0081 ± 0.0162   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'max_sample

# 3/6 - Features: activity, hour, weekday

...

Testing 24 different hyperparameter combinations
 Training Time: 0.8343    Inference Time: 0.4695    pAUC: 0.6676 ± 0.0367   Recall: 0.0364 ± 0.0727   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.4165    Inference Time: 2.0554    pAUC: 0.7166 ± 0.0173   Recall: 0.1061 ± 0.0692   TN: 846814   FP: 7847    FN: 160   TP: 38    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 18.9287   Inference Time: 9.0102    pAUC: 0.7321 ± 0.0089   Recall: 0.0899 ± 0.0222   TN: 847352   FP: 7309    FN: 180   TP: 18    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 37.5934   Inference Time: 17.7300   pAUC: 0.7252 ± 0.0097   Recall: 0.0717 ± 0.0195   TN: 847276   FP: 7385    FN: 177   TP: 21    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.8589    Inference Time: 0.4776    pAUC: 0.6992 ± 0.0147   Recall: 0.0263 ± 0.0285   TN: 849732   FP: 4929    FN: 187   TP: 11    Params: {'max_sample

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.7716    Inference Time: 0.4363    pAUC: 0.5229 ± 0.0087   Recall: 0.0222 ± 0.0182   TN: 848586   FP: 6075    FN: 191   TP: 7     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 3.6164    Inference Time: 1.7410    pAUC: 0.5246 ± 0.0137   Recall: 0.0152 ± 0.0186   TN: 848586   FP: 6075    FN: 191   TP: 7     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 15.8687   Inference Time: 7.4929    pAUC: 0.5299 ± 0.0115   Recall: 0.0071 ± 0.0141   TN: 848586   FP: 6075    FN: 191   TP: 7     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 31.7223   Inference Time: 14.8625   pAUC: 0.5369 ± 0.0008   Recall: 0.0071 ± 0.0141   TN: 848586   FP: 6075    FN: 191   TP: 7     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.7827    Inference Time: 0.4423    pAUC: 0.5281 ± 0.0151   Recall: 0.0222 ± 0.0182   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'max_sample

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.9716    Inference Time: 0.5165    pAUC: 0.5398 ± 0.0304   Recall: 0.0364 ± 0.0266   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.8856    Inference Time: 2.2971    pAUC: 0.6658 ± 0.0411   Recall: 0.0848 ± 0.0305   TN: 846986   FP: 7675    FN: 176   TP: 22    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 21.9398   Inference Time: 10.1558   pAUC: 0.6959 ± 0.0177   Recall: 0.0919 ± 0.0170   TN: 846228   FP: 8433    FN: 174   TP: 24    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 43.0857   Inference Time: 19.6997   pAUC: 0.7076 ± 0.0140   Recall: 0.1061 ± 0.0202   TN: 846494   FP: 8167    FN: 171   TP: 27    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.9993    Inference Time: 0.5259    pAUC: 0.6047 ± 0.0579   Recall: 0.0424 ± 0.0441   TN: 851432   FP: 3229    FN: 176   TP: 22    Params: {'max_sample

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 28 different hyperparameter combinations
 Training Time: 2.4214    Inference Time: 1.4150    pAUC: 0.6283 ± 0.0000   Recall: 0.0657 ± 0.0000   TN: 847646   FP: 7015    FN: 185   TP: 13    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.5084    Inference Time: 2.6899    pAUC: 0.6746 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 850426   FP: 4235    FN: 190   TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.5534    Inference Time: 3.8886    pAUC: 0.6746 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 850426   FP: 4235    FN: 190   TP: 8     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.6154    Inference Time: 5.0700    pAUC: 0.5774 ± 0.0000   Recall: 0.0657 ± 0.0000   TN: 847646   FP: 7015    FN: 185   TP: 13    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.3566    Inference Time: 1.4257    pAUC: 0.6574 ± 0.0000   Recall: 0.0657 ± 0.0000   TN: 848338   FP: 6323    FN: 185   TP: 13    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Features: activity, weekday

Testing 28 different hyperparameter combinations
 Training Time: 2.4129    Inference Time: 1.4451    pAUC: 0.5129 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.5258    Inference Time: 2.7724    pAUC: 0.5129 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.7875    Inference Time: 3.8611    pAUC: 0.5129 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.7814    Inference Time: 5.1087    pAUC: 0.5129 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.4184    Inference Time: 1.4782    pAUC: 0.5242 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Features: activity, hour, weekday

...

Testing 28 different hyperparameter combinations
 Training Time: 2.2946    Inference Time: 1.5083    pAUC: 0.5210 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 851827   FP: 2834    FN: 194   TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.2928    Inference Time: 2.8054    pAUC: 0.5989 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 848717   FP: 5944    FN: 192   TP: 6     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.2519    Inference Time: 4.0622    pAUC: 0.6793 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 846460   FP: 8201    FN: 186   TP: 12    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.4329    Inference Time: 5.2526    pAUC: 0.5416 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 851827   FP: 2834    FN: 194   TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.3256    Inference Time: 1.5412    pAUC: 0.5994 ± 0.0000   Recall: 0.0859 ± 0.0000   TN: 847817   FP: 6844    FN: 181   TP: 17    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 2.3292    Inference Time: 1.4441    pAUC: 0.4740 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.2286    Inference Time: 3.1841    pAUC: 0.4742 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.2732    Inference Time: 3.9474    pAUC: 0.5266 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.2663    Inference Time: 5.0707    pAUC: 0.5122 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.2570    Inference Time: 1.4821    pAUC: 0.5032 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 2.4604    Inference Time: 1.6202    pAUC: 0.5441 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 847415   FP: 7246    FN: 192   TP: 6     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.4645    Inference Time: 3.0729    pAUC: 0.4890 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846572   FP: 8089    FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.5818    Inference Time: 4.3650    pAUC: 0.5144 ± 0.0000   Recall: 0.0657 ± 0.0000   TN: 846286   FP: 8375    FN: 185   TP: 13    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.4804    Inference Time: 5.7390    pAUC: 0.5312 ± 0.0000   Recall: 0.0253 ± 0.0000   TN: 849698   FP: 4963    FN: 193   TP: 5     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.4527    Inference Time: 1.6647    pAUC: 0.4747 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 847034   FP: 7627    FN: 198   TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 1 different hyperparameter combinations
 Training Time: 1.0415    Inference Time: 1.8117    pAUC: 0.5667 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 846617.0   FP: 8044.0   FN: 180.0   TP: 18.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5667 ± 0.0000
Recall: 0.0909 ± 0.0000


# 2/6 - Features: activity, weekday

Testing 1 different hyperparameter combinations
 Training Time: 0.9914    Inference Time: 1.7154    pAUC: 0.5380 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 846374.0   FP: 8287.0   FN: 190.0   TP: 8.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5380 ± 0.0000
Recall: 0.0404 ± 0.0000


# 3/6 - Features: activity, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 1.3094    Inference Time: 2.3288    pAUC: 0.6642 ± 0.0000   Recall: 0.1313 ± 0.0000   TN: 847238.0   FP: 7423.0   FN: 172.0   TP: 26.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6642 ± 0.0000
Recall: 0.1313 ± 0.0000


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.2581    Inference Time: 2.2395    pAUC: 0.5275 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738.0   FP: 3923.0   FN: 198.0   TP: 0.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5275 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.9001    Inference Time: 3.4056    pAUC: 0.6406 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 846354.0   FP: 8307.0   FN: 180.0   TP: 18.0   Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6406 ± 0.0000
Recall: 0.0909 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION,
    n_jobs=1
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 1 different hyperparameter combinations
 Training Time: 1.0173    Inference Time: 1.7459    pAUC: 0.6765 ± 0.0000   Recall: 0.0657 ± 0.0000   TN: 847646   FP: 7015    FN: 185   TP: 13    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6765 ± 0.0000
Recall: 0.0657 ± 0.0000


# 2/6 - Features: activity, weekday

Testing 1 different hyperparameter combinations
 Training Time: 0.9804    Inference Time: 1.7193    pAUC: 0.5069 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5069 ± 0.0000
Recall: 0.0000 ± 0.0000


# 3/6 - Features: activity, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 1.3138    Inference Time: 2.3153    pAUC: 0.6654 ± 0.0000   Recall: 0.1212 ± 0.0000   TN: 849132   FP: 5529    FN: 174   TP: 24    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6654 ± 0.0000
Recall: 0.1212 ± 0.0000


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.2762    Inference Time: 2.2124    pAUC: 0.5299 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5299 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.8550    Inference Time: 3.3257    pAUC: 0.6285 ± 0.0000   Recall: 0.0808 ± 0.0000   TN: 846534   FP: 8127    FN: 182   TP: 16    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6285 ± 0.0000
Recall: 0.0808 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 80 different hyperparameter combinations
 Training Time: 5.6190    Inference Time: 0.8746    pAUC: 0.5780 ± 0.0000   Recall: 0.0606 ± 0.0000   TN: 847779   FP: 6882    FN: 186   TP: 12    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 4.3787    Inference Time: 0.8719    pAUC: 0.5318 ± 0.0000   Recall: 0.0455 ± 0.0000   TN: 849047   FP: 5614    FN: 189   TP: 9     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 6.8556    Inference Time: 0.8725    pAUC: 0.6535 ± 0.0000   Recall: 0.0707 ± 0.0000   TN: 847226   FP: 7435    FN: 184   TP: 14    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 6.8614    Inference Time: 0.7968    pAUC: 0.7230 ± 0.0000   Recall: 0.0808 ± 0.0000   TN: 848129   FP: 6532    FN: 182   TP: 16    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 4.3978    Inference Time: 0.8525    pAUC: 0.5780 ± 0.0000

# 2/6 - Features: activity, weekday

Testing 80 different hyperparameter combinations
 Training Time: 2.1462    Inference Time: 0.3363    pAUC: 0.5069 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.1408    Inference Time: 0.3396    pAUC: 0.4934 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 846374   FP: 8287    FN: 190   TP: 8     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 3.1130    Inference Time: 0.3457    pAUC: 0.5063 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 3.1491    Inference Time: 0.3370    pAUC: 0.5275 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.1481    Inference Time: 0.3416    pAUC: 0.5069 ± 0.0000

# 3/6 - Features: activity, hour, weekday

...

Testing 80 different hyperparameter combinations
 Training Time: 2.3475    Inference Time: 0.3436    pAUC: 0.6031 ± 0.0000   Recall: 0.0556 ± 0.0000   TN: 847153   FP: 7508    FN: 187   TP: 11    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.3718    Inference Time: 0.3470    pAUC: 0.6031 ± 0.0000   Recall: 0.0556 ± 0.0000   TN: 847153   FP: 7508    FN: 187   TP: 11    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 3.4260    Inference Time: 0.3556    pAUC: 0.5772 ± 0.0518   Recall: 0.0444 ± 0.0222   TN: 847153   FP: 7508    FN: 187   TP: 11    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 3.4045    Inference Time: 0.3450    pAUC: 0.6008 ± 0.0048   Recall: 0.0626 ± 0.0141   TN: 847153   FP: 7508    FN: 187   TP: 11    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.3934    Inference Time: 0.3501    pAUC: 0.6031 ± 0.0000

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 2.2947    Inference Time: 0.3541    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.2454    Inference Time: 0.3606    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 3.4269    Inference Time: 0.4161    pAUC: 0.4776 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 3.3846    Inference Time: 0.3548    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.2773    Inference Time: 0.3554    pAUC: 0.4737 ± 0.0000

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 3.3737    Inference Time: 0.3844    pAUC: 0.6031 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 846154   FP: 8507    FN: 190   TP: 8     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 3.3570    Inference Time: 0.3860    pAUC: 0.6033 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 846154   FP: 8507    FN: 190   TP: 8     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.8732    Inference Time: 0.3831    pAUC: 0.6273 ± 0.0000   Recall: 0.0758 ± 0.0000   TN: 848977   FP: 5684    FN: 183   TP: 15    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.8428    Inference Time: 0.3827    pAUC: 0.5687 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 846302   FP: 8359    FN: 194   TP: 4     Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 3.3599    Inference Time: 0.3900    pAUC: 0.6031 ± 0.0000

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 168 different hyperparameter combinations
 Training Time: 0.5561    Inference Time: 0.3043    pAUC: 0.6024 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847809   FP: 6852    FN: 180   TP: 18    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.5588    Inference Time: 0.2991    pAUC: 0.6024 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847809   FP: 6852    FN: 180   TP: 18    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.5514    Inference Time: 0.3022    pAUC: 0.6024 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847809   FP: 6852    FN: 180   TP: 18    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.5535    Inference Time: 0.3051    pAUC: 0.6024 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847809   FP: 6852    FN: 180   TP: 18    Params: {'n_components': 1, 'stan

# 2/6 - Features: activity, weekday

Testing 168 different hyperparameter combinations
 Training Time: 0.5488    Inference Time: 0.3063    pAUC: 0.4937 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.5679    Inference Time: 0.3042    pAUC: 0.4937 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.5615    Inference Time: 0.3067    pAUC: 0.4937 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.5490    Inference Time: 0.3106    pAUC: 0.4937 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'stan

# 3/6 - Features: activity, hour, weekday

...

Testing 168 different hyperparameter combinations
 Training Time: 0.6174    Inference Time: 0.3128    pAUC: 0.5737 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847576   FP: 7085    FN: 180   TP: 18    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.6228    Inference Time: 0.3135    pAUC: 0.5737 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847576   FP: 7085    FN: 180   TP: 18    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.6237    Inference Time: 0.3131    pAUC: 0.5737 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847576   FP: 7085    FN: 180   TP: 18    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.6244    Inference Time: 0.3118    pAUC: 0.5737 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 847576   FP: 7085    FN: 180   TP: 18    Params: {'n_components': 1, 'stan

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.6068    Inference Time: 0.3264    pAUC: 0.4941 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.6118    Inference Time: 0.3178    pAUC: 0.4941 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.6242    Inference Time: 0.3182    pAUC: 0.4941 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.6151    Inference Time: 0.3229    pAUC: 0.4941 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'n_components': 1, 'stan

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.8146    Inference Time: 0.3423    pAUC: 0.5403 ± 0.0000   Recall: 0.0354 ± 0.0000   TN: 846300   FP: 8361    FN: 191   TP: 7     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.8255    Inference Time: 0.3446    pAUC: 0.5403 ± 0.0000   Recall: 0.0354 ± 0.0000   TN: 846300   FP: 8361    FN: 191   TP: 7     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.8243    Inference Time: 0.3420    pAUC: 0.5403 ± 0.0000   Recall: 0.0354 ± 0.0000   TN: 846300   FP: 8361    FN: 191   TP: 7     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.8234    Inference Time: 0.3410    pAUC: 0.5403 ± 0.0000   Recall: 0.0354 ± 0.0000   TN: 846300   FP: 8361    FN: 191   TP: 7     Params: {'n_components': 1, 'stan

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 121.4053   Inference Time: 35.8314   pAUC: 0.6764 ± 0.0246   Recall: 0.0657 ± 0.0000   TN: 848338   FP: 6323    FN: 185   TP: 13    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 155.9355   Inference Time: 33.2565   pAUC: 0.6758 ± 0.0267   Recall: 0.0657 ± 0.0000   TN: 848081   FP: 6580    FN: 185   TP: 13    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 179.1580   Inference Time: 33.5788   pAUC: 0.6621 ± 0.0055   Recall: 0.0657 ± 0.0000   TN: 848088   FP: 6573    FN: 185   TP: 13    Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 171.73

# 2/6 - Features: activity, weekday

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 110.5490   Inference Time: 33.0157   pAUC: 0.4773 ± 0.0073   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 125.3955   Inference Time: 29.7292   pAUC: 0.4776 ± 0.0079   Recall: 0.0081 ± 0.0162   TN: 846374   FP: 8287    FN: 190   TP: 8     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 177.7699   Inference Time: 38.8111   pAUC: 0.5069 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 187.62

# 3/6 - Features: activity, hour, weekday

...

Testing 9 different hyperparameter combinations
 Training Time: 160.5619   Inference Time: 36.3881   pAUC: 0.6186 ± 0.0127   Recall: 0.0828 ± 0.0113   TN: 846798   FP: 7863    FN: 181   TP: 17    Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 179.7790   Inference Time: 39.6826   pAUC: 0.5849 ± 0.0141   Recall: 0.0859 ± 0.0078   TN: 847819   FP: 6842    FN: 181   TP: 17    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 116.9081   Inference Time: 32.0780   pAUC: 0.6005 ± 0.0163   Recall: 0.0818 ± 0.0020   TN: 846798   FP: 7863    FN: 181   TP: 17    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 199.1578   Inference Time: 31.2490   pAUC: 0.6368 ± 0.0001   Recall: 0.0657 ± 0.0000   TN: 847490   FP: 7171    FN: 185   TP: 13    Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 187.9306   Inference Time: 38.3020   pAUC: 0.6411 ± 0.0089   

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 193.6523   Inference Time: 31.1910   pAUC: 0.5141 ± 0.0089   Recall: 0.0081 ± 0.0162   TN: 850297   FP: 4364    FN: 190   TP: 8     Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 170.7845   Inference Time: 42.5228   pAUC: 0.5151 ± 0.0098   Recall: 0.0000 ± 0.0000   TN: 848879   FP: 5782    FN: 198   TP: 0     Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 187.3249   Inference Time: 68.5992   pAUC: 0.5166 ± 0.0074   Recall: 0.0010 ± 0.0020   TN: 848589   FP: 6072    FN: 197   TP: 1     Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 230.9890   Inference Time: 36.4559   pAUC: 0.5207 ± 0.0113   Recall: 0.0000 ± 0.0000   TN: 850738   FP: 3923    FN: 198   TP: 0     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 192.4688   Inference Time: 36.1458   pAUC: 0.5300 ± 0.0002   

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 193.2233   Inference Time: 33.4073   pAUC: 0.5890 ± 0.0287   Recall: 0.0808 ± 0.0207   TN: 846304   FP: 8357    FN: 174   TP: 24    Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 188.9036   Inference Time: 33.9254   pAUC: 0.5819 ± 0.0149   Recall: 0.0687 ± 0.0061   TN: 846283   FP: 8378    FN: 183   TP: 15    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 222.4763   Inference Time: 29.1025   pAUC: 0.5925 ± 0.0294   Recall: 0.0697 ± 0.0108   TN: 846133   FP: 8528    FN: 186   TP: 12    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 231.8888   Inference Time: 33.2867   pAUC: 0.6339 ± 0.0006   Recall: 0.0949 ± 0.0049   TN: 846215   FP: 8446    FN: 178   TP: 20    Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 195.9321   Inference Time: 36.1331   pAUC: 0.6341 ± 0.0002   

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)